In [5]:
# Progetto Data Science Lab
## Sentiment Analysis Corona Virus
## Giacomo De Cobbi
## Alessandro Pontini
## Kevin Tranchina

In [26]:
import os 
import pandas as pd
from datetime import datetime, timedelta
import nltk
import re ##regulare expression
import string
from nltk.tokenize import BlanklineTokenizer
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import MWETokenizer
from nltk.tokenize import TweetTokenizer
import matplotlib.pyplot as plt
import subprocess
import operator
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\tranchinake\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
subprocess.Popen(r'explorer /select,"C:\Users\tranchinake\Desktop\DataScienceUniversity\LabBusinessAnalytic"')

# Import and Preprocessing Dataset

In [109]:
# imposta il path dove vuoi hai i dati

path = str(r'C:\Users\tranchinake\Desktop\DataScienceUniversity\LabBusinessAnalytic')
path = path.replace("\\","/")
os.chdir(path)

In [110]:
# Lettura file 
# 33716 righe
vaccination = pd.read_csv("vaccination_all_tweets.csv")

In [111]:
vaccination.loc[0,'text']

'Same folks said daikon paste could treat a cytokine storm #PfizerBioNTech https://t.co/xeHhIMg1kF'

## Preprocessiong
### Rimozione Hash, rimozione Totale tag e link
#### Esempio con 
'Ciao mi chiamo @Antionio Antonio #Ciao #Home @Nicole https://facebook.com'

In [130]:
esempio = 'Ciao mi chiamo @Antionio Antonio #Ciao #Home @Nicole https://facebook.com'
''.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)","",esempio))

'Ciao mi chiamo  Antonio Ciao Home  '

#### Applicazione al nosto dataset

In [118]:
# rimozione del cancelletto nell'hashtag 
# rimozione dei link e dei tag
vaccination.loc[:,'text'] = vaccination.loc[:,'text'].apply(lambda x : ''.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)","",x)))
vaccination.loc[0:10,'text'].apply(lambda x : print(x))

Same folks said daikon paste could treat a cytokine storm PfizerBioNTech 
While the world has been on the wrong side of history this year hopefully the biggest vaccination effort weve ev 
coronavirus SputnikV AstraZeneca PfizerBioNTech Moderna Covid19 Russian vaccine is created to last 24 years 
Facts are immutable Senator even when youre not ethically sturdy enough to acknowledge them 1 You were born i 
Explain to me again why we need a vaccine   whereareallthesickpeople PfizerBioNTech 
Does anyone have any useful adviceguidance for whether the COVID vaccine is safe whilst breastfeeding 
it is a bit sad to claim the fame for success of vaccination on patriotic competition between USA Canada UK and 
There have not been many bright days in 2020 but here are some of the best 1 BidenHarris winning Election2020 
Covid vaccine You getting it CovidVaccine covid19 PfizerBioNTech Moderna
CovidVaccine States will start getting COVID19Vaccine Monday US says pakustv NYC Healthcare GlobalGoals 
wh

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
Name: text, dtype: object

In [121]:
## lowerCase
# vaccination.text = vaccination.text.str.lower()

## rimozione dei numeri
vaccination.text = vaccination.text.replace(r'\d+','')

## rimozione spazi bianchi
vaccination.text  = vaccination.text.astype(str).str.strip()

# rimozione punteggiatura
# verificato rimuove anche i cancelletti
vaccination.text=vaccination.text.astype(str).str.translate(str.maketrans('','',string.punctuation))

#rimozione \n
vaccination.text = vaccination.text.replace("\n"," ")

#rimozione apice strano
vaccination.text = vaccination.text.replace("'’'"," ")

In [9]:
# TweetTokenizer
tknzr = TweetTokenizer()
vaccination['tweet_tokenize'] = vaccination['text'].apply(tknzr.tokenize)
vaccination['tweet_tokenize']

0        [Same, folks, said, daikon, paste, could, trea...
1        [While, the, world, has, been, on, the, wrong,...
2        [#coronavirus, #SputnikV, #AstraZeneca, #Pfize...
3        [Facts, are, immutable, ,, Senator, ,, even, w...
4        [Explain, to, me, again, why, we, need, a, vac...
                               ...                        
33712    [#BREAKING, The, information, attack, targetin...
33713    [Pity, ,, as, my, personal, preference, would,...
33714    [#WHO, in, close, contact, with, #Russia, on, ...
33715    [#URGENT, |, US, targets, Europe, in, smear, c...
33716    [#BREAKING, #US, Targets, Europe, in, Smear, C...
Name: tweet_tokenize, Length: 33717, dtype: object

In [41]:
# Rimozione Stop Words
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')
stop_words += ['im','youre','thats','get','cant','dont','go',
               'going','gonna','like','got', 'come', 'look','see','theres',
               'say','want','wanna','make','yes','right','']


vaccination['tweet_without_stopwords'] = vaccination['tweet_tokenize'].apply(lambda x: [item for item in x if item not in stop_words])
vaccination['tweet_without_stopwords'] 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tranchinake\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0        [folks, said, daikon, paste, could, treat, cyt...
1        [world, wrong, side, history, year, hopefully,...
2        [coronavirus, sputnikv, astrazeneca, pfizerbio...
3        [facts, immutable, senator, even, ethically, s...
4        [explain, need, vaccine, borisjohnson, matthan...
                               ...                        
33712    [breaking, information, attack, targeting, spu...
33713    [pity, personal, preference, would, astrazenec...
33714    [close, contact, russia, sputnikv, certificati...
33715    [urgent, us, targets, europe, smear, campaign,...
33716    [breaking, us, targets, europe, smear, campaig...
Name: tweet_without_stopwords, Length: 33717, dtype: object

# Sentiment Analysis
## VADER ( Valence Aware Dictionary for Sentiment Reasoning)
#### VADER  is a model used for text sentiment analysis that is sensitive to both polarity (positive/negative) and intensity (strength) of emotion. It is available in the NLTK package and can be applied directly to unlabeled text data.

#### To calculate the sentimental score of the entire text, Vader scans the text for known sentimental features, modified the intensity and polarity according to the rules, summed up the scores of features found within the text and normalized the final score to (-1, 1) using function:



$$ compound=\frac{x}{\sqrt{x^2 +a}}$$

 

#### In Vader, alpha is set to be 15 which approximates the maximum expected value of x.

In [11]:
vaccination['scores']  = vaccination['text'] .apply(lambda review: sid.polarity_scores(review))

In [12]:
vaccination['scores']

0        {'neg': 0.0, 'neu': 0.787, 'pos': 0.213, 'comp...
1        {'neg': 0.125, 'neu': 0.766, 'pos': 0.109, 'co...
2        {'neg': 0.0, 'neu': 0.875, 'pos': 0.125, 'comp...
3        {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
4        {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
                               ...                        
33712    {'neg': 0.162, 'neu': 0.838, 'pos': 0.0, 'comp...
33713    {'neg': 0.086, 'neu': 0.914, 'pos': 0.0, 'comp...
33714    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
33715    {'neg': 0.139, 'neu': 0.721, 'pos': 0.14, 'com...
33716    {'neg': 0.152, 'neu': 0.848, 'pos': 0.0, 'comp...
Name: scores, Length: 33717, dtype: object

In [13]:
# Test
a = 'I love my girl'
print(sid.polarity_scores(a))

#-0,5<x<+0.5 

{'neg': 0.0, 'neu': 0.323, 'pos': 0.677, 'compound': 0.6369}


In [14]:
vaccination['compound']  = vaccination['scores'].apply(lambda score_dict: score_dict['compound'])
vaccination['compound']

0        0.4019
1       -0.1027
2        0.2500
3        0.0000
4        0.0000
          ...  
33712   -0.4767
33713   -0.1531
33714    0.0000
33715    0.0085
33716   -0.3612
Name: compound, Length: 33717, dtype: float64

#### Aggregazioni e analisi

In [128]:
print(vaccination.columns)
print(vaccination.info())
print('\nDescribe\n')
print(vaccination['user_location'].describe())

Index(['id', 'user_name', 'user_location', 'user_description', 'user_created',
       'user_followers', 'user_friends', 'user_favourites', 'user_verified',
       'date', 'text', 'hashtags', 'source', 'retweets', 'favorites',
       'is_retweet'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33717 entries, 0 to 33716
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                33717 non-null  int64 
 1   user_name         33717 non-null  object
 2   user_location     25937 non-null  object
 3   user_description  31402 non-null  object
 4   user_created      33717 non-null  object
 5   user_followers    33717 non-null  int64 
 6   user_friends      33717 non-null  int64 
 7   user_favourites   33717 non-null  int64 
 8   user_verified     33717 non-null  bool  
 9   date              33717 non-null  object
 10  text              33717 non-null  object
 11  hashtags          

In [129]:
vaccination = vaccination[vaccination['user_location'].notna()]
#vaccination['user_location2'] = vaccination['user_location'].dropna()
vaccination.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25937 entries, 0 to 33716
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                25937 non-null  int64 
 1   user_name         25937 non-null  object
 2   user_location     25937 non-null  object
 3   user_description  24970 non-null  object
 4   user_created      25937 non-null  object
 5   user_followers    25937 non-null  int64 
 6   user_friends      25937 non-null  int64 
 7   user_favourites   25937 non-null  int64 
 8   user_verified     25937 non-null  bool  
 9   date              25937 non-null  object
 10  text              25937 non-null  object
 11  hashtags          20371 non-null  object
 12  source            25929 non-null  object
 13  retweets          25937 non-null  int64 
 14  favorites         25937 non-null  int64 
 15  is_retweet        25937 non-null  bool  
dtypes: bool(2), int64(6), object(8)
memory usage: 3.0+ MB


In [25]:
vaccination.loc[vaccination['user_location'].str.contains('Italy', regex=True, na=False)]
vaccination.loc[0,'text']

'Same folks said daikon paste could treat a cytokine storm PfizerBioNTech httpstcoxeHhIMg1kF'

In [25]:
vaccination['hashtags']

0                                       ['PfizerBioNTech']
1                                                      NaN
2        ['coronavirus', 'SputnikV', 'AstraZeneca', 'Pf...
3                                                      NaN
4           ['whereareallthesickpeople', 'PfizerBioNTech']
                               ...                        
33712                       ['BREAKING', 'SputnikV', 'US']
33713                 ['AstraZeneca', 'vaccine', 'Pfizer']
33714                        ['WHO', 'Russia', 'SputnikV']
33715                                           ['URGENT']
33716             ['BREAKING', 'US', 'Russia', 'SputnikV']
Name: hashtags, Length: 33717, dtype: object